# Autodiff:  <br> Calculus  from another angle 
(and the special role played by Julia's multiple dispatch and compiler technology)

(Alan Edelman, 2017)


  The first time I heard about automatic differentiation, it was easy for me to imagine what it was.  I was wrong.  In my head, I thought it was straightforward symbolic differentiation applied to code.  I kind of imagined it was like executing Mathematica or Maple, or even just automatically doing what I learned to do in my calculus class. 
  <img src="derfunc.gif" width="230">
  .... and anyway if it was not that, then it must be finite differences, like one learns in a numerical computing class.
  
<img src="http://image.mathcaptain.com/cms/images/122/Diff%202.png" width="150">



## Babylonian sqrt

I would like to use a simple example, computation of sqrt(x), where for me how autodiff works came as both a mathematical surprise, and a computing wonder.  The example is  the Babylonian algorithm, known to man for millenia, to compute sqrt(x):  


 > Repeat $ t \leftarrow  (t+x/t) / 2 $ until $t$ converges to $\sqrt{x}$.
 
 Each iteration has one add and two divides. For illustration purposes, 10 iterations suffice.

In [2]:
function Babylonian(x; N = 10) 
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end    
    t
end  

Babylonian (generic function with 1 method)

In [4]:
Babylonian(π), √π   # \pi + <tab> , and \sqrt + <tab> 

(1.7724538509055159, 1.7724538509055159)

Check that it works:

In [ ]:
x=2; Babylonian(x),√x  # Type \sqrt+<tab> to get the symbol

In [5]:
using Plots
plotly()

Plots.PlotlyBackend()

In [6]:
i = 0:.01:49

plot([x->Babylonian(x,N=i) for i=1:5],i,label=["Iteration $j" for i=1:1,j=1:5])

plot!(sqrt,i,c="black",label="sqrt",
      title = "Those Babylonians really knew how to √")

## ...and now the derivative, almost by magic

In [8]:
struct D <: Number  # D is a function-derivative pair
    f::Tuple{Float64,Float64}
end

Sum Rule: (x+y)' = x' + y' <br>
Quotient Rule: (x/y)' = (yx'-xy') / y^2

In [9]:
import Base: +, /, convert, promote_rule
+(x::D, y::D) = D(x.f .+ y.f)
/(x::D, y::D) = D((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
convert(::Type{D}, x::Real) = D((x,zero(x)))
promote_rule(::Type{D}, ::Type{<:Number}) = D

promote_rule (generic function with 140 methods)

In [10]:
x=2; Babylonian(D((x,1))),(√x,.5/√x)

(D((1.414213562373095, 0.35355339059327373)), (1.4142135623730951, 0.35355339059327373))

In [11]:
i = .2:.01:49
plot([x->Babylonian(D((x,1.0)),N=i).f[2] for i=1:5],i)
plot!(x->.5/√x,i,c="black",label="d(sqrt(x))/dx",
    title = " Babylonians Differentiated")

## It just works!

How does it work?  We will explain in a moment.  Right now marvel that it does.  Note we did not
import any autodiff package.  Everything is just basic vanilla Julia.

## The assembler

Most folks don't read assembler, but one can see that it is short.

In [12]:
@inline function Babylonian(x; N = 10) 
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end    
    t
end  
@code_native(Babylonian(D((2,1))))

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[12]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	movsd	(%rsi), %xmm0           ## xmm0 = mem[0],zero
	movsd	8(%rsi), %xmm8          ## xmm8 = mem[0],zero
	movabsq	$4889889024, %rax       ## imm = 0x12375C900
	movsd	(%rax), %xmm4           ## xmm4 = mem[0],zero
	addsd	%xmm0, %xmm4
	xorpd	%xmm2, %xmm2
	movapd	%xmm8, %xmm1
	addsd	%xmm2, %xmm1
	addsd	%xmm1, %xmm1
	mulsd	%xmm4, %xmm2
	subsd	%xmm2, %xmm1
	movabsq	$4889889032, %rcx       ## imm = 0x12375C908
	mulsd	(%rcx), %xmm4
	movabsq	$4889889040, %rdx       ## imm = 0x12375C910
	mulsd	(%rdx), %xmm1
	movl	$9, %eax
	movabsq	$4889889048, %rsi       ## imm = 0x12375C918
	movsd	(%rsi), %xmm9           ## xmm9 = mem[0],zero
	movsd	(%rcx), %xmm5           ## xmm5 = mem[0],zero
	movsd	(%rdx), %xmm6           ## xmm6 = mem[0],zero
	nopw	%cs:(%rax,%rax)
Source line: 17
L128:
	movapd	%xmm0, %xmm7
	divsd	%xmm4, %xmm7
	movapd	%xmm4, %xmm2
	mulsd	%xmm8, %xmm2
	movapd	%xmm1, %xmm3
	mulsd	%xm

## Symbolically

We haven't yet explained how it works, but it may be of some value to understand that the below is mathematically
equivalent, though not what the computation is doing.

Notice in the below that Babylonian works on SymPy symbols.

In [13]:
#Pkg.add("SymPy")
using SymPy                    

In [14]:
x = symbols("x")
display("Iterations as a function of x")
for k = 1:5
 display( simplify(Babylonian(x,N=k)))
end

display("Derivatives as a function of x")
for k = 1:5
 display(simplify(diff(simplify(Babylonian(x,N=k)),x)))
end

"Iterations as a function of x"

x   1
- + -
2   2

           2
    (x + 1) 
x + --------
       4    
------------
   x + 1    

 4       3       2           
x  + 28*x  + 70*x  + 28*x + 1
-----------------------------
     / 3      2          \   
   8*\x  + 7*x  + 7*x + 1/   

 8        7         6         5          4         3         2            
x  + 120*x  + 1820*x  + 8008*x  + 12870*x  + 8008*x  + 1820*x  + 120*x + 1
--------------------------------------------------------------------------
         / 7       6        5        4        3        2           \      
      16*\x  + 35*x  + 273*x  + 715*x  + 715*x  + 273*x  + 35*x + 1/      

...

1/2

   2            
  x  + 2*x + 5  
----------------
  / 2          \
4*\x  + 2*x + 1/

 6       5        4        3        2             
x  + 14*x  + 147*x  + 340*x  + 375*x  + 126*x + 21
--------------------------------------------------
  / 6       5       4        3       2           \
8*\x  + 14*x  + 63*x  + 100*x  + 63*x  + 14*x + 1/

 14       13         12          11           10            9            8    
x   + 70*x   + 3199*x   + 52364*x   + 438945*x   + 2014506*x  + 5430215*x  + 8
------------------------------------------------------------------------------
      / 14       13         12          11           10           9           
   16*\x   + 70*x   + 1771*x   + 20540*x   + 126009*x   + 440986*x  + 920795*x

        7            6            5            4           3          2       
836200*x  + 8842635*x  + 5425210*x  + 2017509*x  + 437580*x  + 52819*x  + 3094
------------------------------------------------------------------------------
8            7           6           5           4          3         2       
  + 1173960*x  + 920795*x  + 440986*x  + 126009*x  + 20540*x  + 1771*x  + 70*x

       
*x + 85
-------
    \  
 + 1/  

 30        29          28            27              26               25      
x   + 310*x   + 59799*x   + 4851004*x   + 215176549*x   + 5809257090*x   + 102
------------------------------------------------------------------------------
                     / 30        29          28            27             26  
                  32*\x   + 310*x   + 36611*x   + 2161196*x   + 73961629*x   +

           24                  23                   22                   21   
632077611*x   + 1246240871640*x   + 10776333438765*x   + 68124037776390*x   + 
------------------------------------------------------------------------------
             25                24                 23                  22      
 1603620018*x   + 23367042639*x   + 238538538360*x   + 1758637118685*x   + 957

                 20                     19                     18             
321156247784955*x   + 1146261110726340*x   + 3133113888931089*x   + 6614351291
--------------------------------------------------

## How autodiff is getting the answer
Let us by hand take the "derivative" of the Babylonian iteration with respect to x. Specifically t′=dt/dx

In [16]:
function dBabylonian(x; N = 10) 
    t = (1+x)/2
    t′ = 1/2
    for i = 1:N;  
        t = (t+x/t)/2; 
        t′= (t′+(t-x*t′)/t^2)/2; 
    end    
    t′
end  

dBabylonian (generic function with 1 method)

In [17]:
x = π; dBabylonian(x), .5/√x

(0.2820947917738782, 0.28209479177387814)

What just happened?  Answer: We created an iteration by hand for t′ given our iteration for t. Then we ran the iteration alongside the iteration for t.

In [ ]:
Babylonian(D((x,1)))

How did this work?  It created the same derivative iteration, using very general rules that are set once and need not be written by hand.

Important:: The derivative is substituted before the JIT compiler, and thus efficient compiled code is executed.

## Dual Number Notation

Instead of D(a,b) we can write a + b ϵ, where ϵ satisfies ϵ^2=0.  (Some people like to recall imaginary numbers where an i is introduced with i^2=-1.) 

Others like to think of how engineers just drop the O(ϵ^2) terms.

The four rules are

$ (a+b\epsilon) \pm (c+d\epsilon) = (a+c) \pm (b+d)\epsilon$

$ (a+b\epsilon) * (c+d\epsilon) = (ac) + (bc+ad)\epsilon$

$ (a+b\epsilon) / (c+d\epsilon) = (a/c) + (bc-ad)/d^2 \epsilon $


In [ ]:
Base.show(io::IO,x::D) = print(io,x.f[1]," + ",x.f[2]," ϵ")

In [18]:
# Add the last two rules
import Base: -,*
-(x::D, y::D) = D(x.f .- y.f)
*(x::D, y::D) = D((x.f[1]*y.f[1], (x.f[2]*y.f[1] + x.f[1]*y.f[2])))

* (generic function with 215 methods)

In [19]:
ϵ  = D((0,1))

D((0.0, 1.0))

In [20]:
@code_native(ϵ^2)

	.section	__TEXT,__text,regular,pure_instructions
Filename: intfuncs.jl
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%rbx
	subq	$24, %rsp
	movq	%rdi, %rbx
Source line: 195
	movabsq	$power_by_squaring, %rax
	leaq	-24(%rbp), %rdi
	callq	*%rax
	movq	-24(%rbp), %rax
	movq	-16(%rbp), %rcx
	movq	%rcx, 8(%rbx)
	movq	%rax, (%rbx)
	movq	%rbx, %rax
	addq	$24, %rsp
	popq	%rbx
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [21]:
ϵ * ϵ 

D((0.0, 0.0))

In [23]:
1/(1+ϵ)  # Exact power series:  1-ϵ+ϵ²-ϵ³-...

D((1.0, -1.0))

In [24]:
(1+ϵ)^10 ## Note this just works (we didn't train powers)!!

D((1.0, 10.0))

## Generalization to arbitrary roots

In [ ]:
function nthroot(x, n=2; t=1, N = 10) 
    for i = 1:N;   t += (x/t^(n-1)-t)/n; end   
    t
end  

In [ ]:
nthroot(2,3), ∛2 # take a cube root

In [ ]:
nthroot(7,12), 7^(1/12)

In [ ]:
x = 17.0
nthroot( x+ϵ,3), ∛x, 1/x^(2/3)/3

## Forward Diff
Now that you understand it, you can use the official package

In [ ]:
using ForwardDiff

In [ ]:
ForwardDiff.derivative(sqrt, 2)

In [ ]:
ForwardDiff.derivative(Babylonian, 2)

In [ ]:
@which ForwardDiff.derivative(sqrt, 2)

## Close Look at Convergence

In [ ]:
setprecision(3000)
round.(Float64.(log10.([Babylonian(BigFloat(2),N=k) for k=1:10] - √BigFloat(2))),3)

In [ ]:
struct D1{T} <: Number  # D is a function-derivative pair
    f::Tuple{T,T}
end

In [ ]:
z = D((2.0,1.0))
z1 = D1((BigFloat(2.0),BigFloat(1.0)))

In [ ]:
import Base: +, /, convert, promote_rule
+(x::D1, y::D1) = D1(x.f .+ y.f)
/(x::D1, y::D1) = D1((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
convert(::Type{D1{T}}, x::Real) where {T} = D1((convert(T, x), zero(T)))
promote_rule(::Type{D1{T}}, ::Type{S}) where {T,S<:Number} = D1{promote_type(T,S)}

In [ ]:
A = randn(3,3)

In [ ]:
x = randn(3)

In [ ]:
ForwardDiff.gradient(x->x'A*x,x)

In [ ]:
(A+A')*x